# basicsynbio designs

This notebook contains a collection of basicsynbio designs relevant to interfacing bioCRNpyler and basicynbio.

## Thoughts and questions

- What parameters will BioCRNPyler calculate in predicting constitutive expression from inducible expression results?

## Aims and objectives for cell/s below

- Can I change the orientation of expression casettes?

In [1]:
import basicsynbio as bsb
from basicsynbio.main import DEFAULT_ANNOTATIONS
from basicsynbio.cam import seqrecord_hexdigest
from pathlib import Path
import re

In [2]:
parts = {
    "cds": bsb.BASIC_CDS_PARTS["v0.1"]["sfGFP"],
    "promoter": bsb.BASIC_PROMOTER_PARTS["v0.1"]["B-P3"]
}
reversed_parts = {}
for key, value in parts.items():
    reversed_assembly = bsb.BasicAssembly(
        f"{value.id}_reversed",
        bsb.BASIC_SEVA_PARTS["v0.1"]["18"],
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMS"],
        value,
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMP"],    
    )
    reversed_part = reversed_assembly.return_part(
        name=f"{value.name} reversed",
    )
    reversed_part.id = seqrecord_hexdigest(reversed_part)
    reversed_parts[key] = reversed_part
reverse_construct = bsb.BasicAssembly(
    "reverse_sfGFP_cassette",
    bsb.BASIC_SEVA_PARTS["v0.1"]["26"],
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMS"],
    reversed_parts["cds"],
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["UTR1-RBS1"],
    reversed_parts["promoter"],
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMP"],        
)
path_to_seqs = Path.cwd().parents[0] / "sequences"
# bsb.export_sequences_to_file(
#     reverse_construct.return_part(),
#     path_to_seqs / "reverse_sfGFP_cassette.gb",
#     "genbank",
# )

## Results from above cells

- Can't change the orientation of an expression cassette.

## Aims and objectives for cell/s below

- Design the following constructs that enable:
    - Single plasmid containing IPTG-inducible sfGFP and ATC-inducible mTagBFP2. The reason for mTagBFP2 over mCherry is that it causes more burden and therefore should strongly influence sfGFP expression.
    - Multiple plasmids where sfGFP is expressed from each relevant constitutive promoter.
    - Multiple plasmids where mTagBFP2 is expressed from each relevant constitutive promoter.

In [16]:
inducible_con = bsb.BasicAssembly(
    "inducible_sfGFP_mTagBFP2",
    bsb.BASIC_SEVA_PARTS["v0.1"]["26"],
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMP"],
    bsb.BASIC_PROMOTER_PARTS["v0.1"]["B-P31"],
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["UTR1-RBS3"],
    bsb.BASIC_CDS_PARTS["v0.1"]["sfGFP"],
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["L1"],
    bsb.BASIC_PROMOTER_PARTS["v0.1"]["B-P35"],
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["UTR2-RBS3"],
    bsb.BASIC_CDS_PARTS["v0.1"]["mTagBFP2"],    
    bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMS"]
)
sfgfp_cons = [
    bsb.BasicAssembly(
        f"sfGFP_cassette_{promoter.name}",
        bsb.BASIC_SEVA_PARTS["v0.1"]["26"],
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMP"],
        promoter,
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["UTR1-RBS3"],
        bsb.BASIC_CDS_PARTS["v0.1"]["sfGFP"],
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMS"]
    )
for promoter in bsb.BASIC_PROMOTER_PARTS["v0.1"].values() if re.match("B-P(\d)*_Terminator1_J23(\d)*_RiboA", promoter.name)]
mtagbfp2_cons = [
    bsb.BasicAssembly(
        f"sfGFP_cassette_{promoter.name}",
        bsb.BASIC_SEVA_PARTS["v0.1"]["26"],
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMP"],
        promoter,
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["UTR2-RBS3"],
        bsb.BASIC_CDS_PARTS["v0.1"]["mTagBFP2"],
        bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]["LMS"]
    )
for promoter in bsb.BASIC_PROMOTER_PARTS["v0.1"].values() if re.match("B-P(\d)*_Terminator2_J23(\d)*_RiboB", promoter.name)]
build_1 = bsb.BasicBuild(
    inducible_con,
    *sfgfp_cons,
    *mtagbfp2_cons
)
bsb.export_sequences_to_file(
    build_1.basic_assemblies,
    path_to_seqs / "build_1.gb",
    "genbank"
)

/home/hainesm6/.pyenv/versions/generic_385/lib/python3.8/site-packages/Bio/SeqIO/InsdcIO.py:721: BiopythonWarning: Increasing length of locus line to allow long name. This will result in fields that are not in usual positions.
  warnings.warn(


In [17]:
print(f"Number of assemblies to construct {len(build_1.basic_assemblies)}")

Number of assemblies to construct 17
Number of sfGFP assemblies 8
